In [2]:
import pandas as pd
import geopandas as gpd
import os
import glob
import string

In [3]:
path =(r"C:/projectos/shapes/marcos_clasificacion/")
# Change the directory
os.chdir(path)

In [3]:
dict_shapes = {}

# Iterate over all the files in the directory
for file in os.listdir():
   if file.endswith('.shp') and file[0].isalpha():
       temp = gpd.read_file(file)
       dict_shapes[file] = temp

In [4]:
dict_shapes["Albacete_class.shp"].head(2)

,AREA,REFCAT,A_TOT_EDIF,SUP_ED_0,SUP_ED_1,MAX_PLANTA,NUM_INM_R,USO,PROP_AE_AT,RES_UNI,...,IND,OFI,COM,EQUIP_SANI,EQUIP_EDU,EQUIP_OTR,OCIO_ESP,HOS_REST,ED_SING,geometry
0,472,6978915WJ9167H,0,0,0,0,0,SIN_EDIF,0.000000,0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((596928.841 4317779.775, 596895.924 4..."
1,2138,6879902WJ9167H,5918,939,1177,4,29,RES_PLU,0.550514,0,...,0,0,0,0,0,1,0,0,0,"POLYGON ((596875.544 4317938.072, 596849.645 4..."


In [5]:
clases_agregadas = pd.read_csv("C:/projectos/shapes/clases_agregadas.csv")
clases_agregadas

,USO,USO_AG
0,COM,COM
1,ED_SING,ED_SING
2,EQUIP_EDU,EQUIP
3,EQUIP_OTR,EQUIP
4,EQUIP_SANI,EQUIP
5,HOS_REST,OCIO
6,IND,IND
7,IND_MX,IND
8,OCIO_ESP,OCIO
9,OFI,OFI


In [6]:
def uso_agregado_gtp(data_in, clases):
    data_out = {}
    for key, value in data_in.items():
        data_out[key] = value.merge(clases, on = 'USO', how = 'left')
    return data_out

In [7]:
def uso_agregado(data_in,data_out,clases):
    for keys, values in data_in.items():
        data_out[keys]= pd.merge(data_in[keys], clases, on ='USO', how = "left")
    return(data_out)

In [10]:
uso_agregado_gtp = uso_agregado_gtp(dict_shapes,clases_agregadas)

In [11]:
## Crea un código único para los usos agregados

import numpy as np
import pandas as pd

def asignacion_codigo_categorias_chatgtp(data_in):
    data_out = {}
    for key, df in data_in.items():
        df = df.reset_index()
        usos = df['USO_AG'].unique()
        cod_usos = np.arange(len(usos))
        cod_map = pd.Series(cod_usos, index=usos)
        df['cod_usos_ag'] = df['USO_AG'].map(cod_map)
        data_out[key] = df
    return data_out

In [12]:
asignacion_usos_agregados = asignacion_codigo_categorias_chatgtp(uso_agregado_gtp)

In [13]:
asignacion_usos_agregados["Albacete_class.shp"].head(2)

,index,AREA,REFCAT,A_TOT_EDIF,SUP_ED_0,SUP_ED_1,MAX_PLANTA,NUM_INM_R,USO,PROP_AE_AT,...,COM,EQUIP_SANI,EQUIP_EDU,EQUIP_OTR,OCIO_ESP,HOS_REST,ED_SING,geometry,USO_AG,cod_usos_ag
0,0,472,6978915WJ9167H,0,0,0,0,0,SIN_EDIF,0.000000,...,0,0,0,0,0,0,0,"POLYGON ((596928.841 4317779.775, 596895.924 4...",SIN_EDIF,0
1,1,2138,6879902WJ9167H,5918,939,1177,4,29,RES_PLU,0.550514,...,0,0,0,1,0,0,0,"POLYGON ((596875.544 4317938.072, 596849.645 4...",RES_PLU,1
2,2,400,6978903WJ9167H,0,0,0,0,0,SIN_EDIF,0.000000,...,0,0,0,0,0,0,0,"POLYGON ((597012.007 4317902.184, 597000.745 4...",SIN_EDIF,0
3,3,647,6409719WJ9260N,809,648,0,0,0,IND,1.001546,...,0,0,0,0,0,0,0,"POLYGON ((596325.016 4320621.310, 596299.126 4...",IND,2
4,4,250,3984813WJ9038S,275,95,100,1,2,RES_UNI,0.400000,...,0,0,0,0,0,0,0,"POLYGON ((593827.090 4308198.986, 593807.284 4...",RES_UNI,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16335,16335,9510,7658912WJ9175N,0,0,0,0,0,SIN_EDIF,0.000000,...,0,0,0,0,0,0,0,"POLYGON ((597374.826 4315369.379, 597374.950 4...",SIN_EDIF,0
16336,16336,1445,2239008WJ9223N,257,257,0,0,2,RES_UNI,0.177855,...,0,0,0,0,0,0,0,"POLYGON ((592132.399 4323711.287, 592124.179 4...",RES_UNI,3
16337,16337,198,8478011WJ9187G,887,199,172,4,5,RES_PLU_MX,1.005051,...,20,0,0,0,0,0,0,"POLYGON ((598305.200 4317610.051, 598305.686 4...",RES_PLU,1
16338,16338,176,7662322WJ9176D,0,0,0,0,0,SIN_EDIF,0.000000,...,0,0,0,0,0,0,0,"POLYGON ((597514.170 4316059.622, 597507.516 4...",SIN_EDIF,0


In [15]:
asignacion_usos_agregados["Albacete_class.shp"].to_file('C:/projectos/shapes/test/albacete.shp')

C:\Users\crist\AppData\Local\Temp\ipykernel_9332\110754505.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  asignacion_usos_agregados["Albacete_class.shp"].to_file('C:/projectos/shapes/test/albacete.shp')


In [14]:
asignacion_usos_agregados["Albacete_class.shp"][asignacion_usos_agregados["Albacete_class.shp"].loc[:,"USO_AG"] == 'nan']

,index,AREA,REFCAT,A_TOT_EDIF,SUP_ED_0,SUP_ED_1,MAX_PLANTA,NUM_INM_R,USO,PROP_AE_AT,...,COM,EQUIP_SANI,EQUIP_EDU,EQUIP_OTR,OCIO_ESP,HOS_REST,ED_SING,geometry,USO_AG,cod_usos_ag


In [16]:
for keys, values in asignacion_usos_agregados.items():
    asignacion_usos_agregados[keys].to_file('C:/projectos/shapes/marcos_clasificacion_ag/'+ keys)

C:\Users\crist\AppData\Local\Temp\ipykernel_9332\1833407407.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  asignacion_usos_agregados[keys].to_file('C:/projectos/shapes/marcos_clasificacion_ag/'+ keys)
C:\Users\crist\AppData\Local\Temp\ipykernel_9332\1833407407.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  asignacion_usos_agregados[keys].to_file('C:/projectos/shapes/marcos_clasificacion_ag/'+ keys)
C:\Users\crist\AppData\Local\Temp\ipykernel_9332\1833407407.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  asignacion_usos_agregados[keys].to_file('C:/projectos/shapes/marcos_clasificacion_ag/'+ keys)
C:\Users\crist\AppData\Local\Temp\ipykernel_9332\1833407407.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  asignacion_usos_agregados[keys].to_file('C:/pr